In [31]:
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import pathlib
import datetime
from math import ceil
from src.data.dataset_loader import load_distracted_driver_detection_list, load_dataset_image_label
from src.data.image_label_loader import ImageLabelLoader
from src.evaluation.metrics import accuracy, confusion_matrix
from src.utils.data_util import get_paths, sample_dataset
from src.preprocessing.standardizer import Standardizer
from src.visualization.history_plotter import plot_loss_acc_history_epoch
from src.visualization.weights_visualization import visualize_weights
from src.solver.tensorflow_solver import TensorflowSolver

# Load tensorboard notebook extenstion 
%load_ext tensorboard

%load_ext autoreload
%autoreload 2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_dir = pathlib.Path("dataset/raw/imgs/train") # Train directory

X_train_filenames, X_val_filenames, y_train_labels, y_val_labels = load_distracted_driver_detection_list(val_size=0.2, split_on_driver=True, random_state=12)
train_paths = get_paths(data_dir, X_train_filenames, y_train_labels)
val_paths = get_paths(data_dir, X_val_filenames, y_val_labels)
class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))

num_train = len(train_paths)
num_val = len(val_paths)
num_classes = len(class_names)

print(f'Training set size: {num_train}')
print(f'Validation set size: {num_val}')
print(f'Number of classes: {num_classes}')

Training set size: 17761
Validation set size: 4663
Number of classes: 10


In [3]:
CPU = '/cpu:0'
GPU = '/device:GPU:0'
physical_GPU = tf.config.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(physical_GPU, True)

In [4]:
rng = tf.random.Generator.from_seed(123, alg='philox')

def wrap_augment(image, label):
    seed = rng.make_seeds(2)[0]
    image = augment(image, seed)
    return image, label

def augment(image, seed):
    image = tf.image.stateless_random_brightness(image, max_delta=0.5, seed=seed)
    image = tf.image.stateless_random_hue(image, max_delta=0.5, seed=seed)
    image = tf.image.stateless_random_contrast(image, lower=0.9, upper=1.2, seed=seed)
    image = tf.image.stateless_random_saturation(image, lower=1, upper=1.2, seed=seed)
    image = tf.image.stateless_random_jpeg_quality(image, min_jpeg_quality=50, max_jpeg_quality=100, seed=seed)
    #image = tf.image.stateless_random_crop(image, size=(60,60,3), seed=seed)
    #image = tf.image.stateless_random_flip_left_right(image, seed)
    #image = tf.image.stateless_random_flip_up_down(image, seed)
    return image

def preprocess(X_batch, y_batch):
    X_batch = standardizer.transform(X_batch) # Standardize
    return X_batch, y_batch

In [6]:
# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del train_dset
   del val_dset
   print('Clear previously loaded data.')
except:
   pass


BATCH_SIZE = 16
IMG_SHAPE = (128, 128, 3)
loader = ImageLabelLoader(class_names, img_shape=IMG_SHAPE)
standardizer = Standardizer()
normalization_samples_batch = ceil((num_train * 0.4) / BATCH_SIZE)

# Force image load and preprocessing with specific device
with tf.device(CPU):
    # Train dataset input pipeline
    train_dset = tf.data.Dataset.from_tensor_slices(train_paths)
    train_dset = train_dset.map(loader.load, num_parallel_calls=tf.data.experimental.AUTOTUNE) # Load from path to image, label
    train_dset = train_dset.map(wrap_augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dset = train_dset.cache()
    train_dset = train_dset.shuffle(buffer_size=num_train, reshuffle_each_iteration=True)
    train_dset = train_dset.batch(BATCH_SIZE, drop_remainder=False)
    standardizer.fit(train_dset, num_samples_batch=normalization_samples_batch) # Fit mean and std to train set
    train_dset = train_dset.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train_dset = train_dset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    # Validation dataset input pipeline
    val_dset = tf.data.Dataset.from_tensor_slices(val_paths)
    val_dset = val_dset.map(loader.load, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dset = val_dset.batch(BATCH_SIZE, drop_remainder=False)
    val_dset = val_dset.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    val_dset = val_dset.cache()
    val_dset = val_dset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [7]:
class ConvNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        initializer = tf.initializers.VarianceScaling(scale=2.0)

        self.conv_1 = tf.keras.layers.Conv2D(64, (3,3), strides=1, padding="same", activation='relu',kernel_initializer=initializer)
        self.batch_1 = tf.keras.layers.BatchNormalization() 

        self.maxpool_1 = tf.keras.layers.MaxPool2D() 

        self.conv_3 = tf.keras.layers.Conv2D(128, (1,1), strides=1, padding="same", activation='relu', kernel_initializer=initializer)
        self.batch_3 = tf.keras.layers.BatchNormalization()

        self.maxpool_2 = tf.keras.layers.MaxPool2D() 
        
        self.flatten = tf.keras.layers.Flatten()
        self.dropout = tf.keras.layers.Dropout(rate=0.7)
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer, kernel_regularizer=tf.keras.regularizers.l2())
    
    def call(self, x, training=False):
        
        x = self.conv_1(x)
        x = self.batch_1(x, training=training)

        x = self.maxpool_1(x)

        x = self.conv_3(x)
        x = self.batch_3(x, training=training)

        x = self.maxpool_2(x)

        x = self.flatten(x)
        x = self.dropout(x, training=training)
        scores = self.fc(x)

        return scores

In [ ]:
lr = 0.01
decay_steps = 1
decay_rate = 0.95

model = ConvNet(num_classes)

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                                    initial_learning_rate=lr,
                                    decay_steps=decay_steps,
                                    decay_rate=decay_rate,
                                    staircase=False)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

solver = TensorflowSolver(model, optimizer, train_dset, val_dset,
                            num_epochs=10,
                            verbose=True)
solver.train(device=GPU)

In [ ]:
plot_loss_acc_history_epoch(solver.loss_history, solver.val_loss_history, solver.train_acc_history, solver.val_acc_history)

In [ ]:
OPTIMIZER = 'adam'
LOSS = 'sparse_categorical_crossentropy'
TENSORBOARD_DIR = 'tensorboard/logs/fit/'

model = ConvNet(num_classes)
model.compile(optimizer=OPTIMIZER,
            loss=LOSS,
            metrics=[tf.keras.metrics.sparse_categorical_accuracy])

# Setup tensorboard
log_dir = TENSORBOARD_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

with tf.device(GPU):
    trainer = model.fit(train_dset, epochs=3, validation_data=val_dset, callbacks=[tensorboard_callback])

In [ ]:
plot_loss_acc_history_epoch(trainer.history['loss'], trainer.history['val_loss'], trainer.history['sparse_categorical_accuracy'], trainer.history['val_sparse_categorical_accuracy'])